In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
reward_name = "OpenAssistant/reward-model-deberta-v3-large-v2"
rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained(reward_name, device_map='cuda'), AutoTokenizer.from_pretrained(reward_name, torch_device="cuda")
question, answer = "Explain nuclear fusion like I am five", "Nuclear fusion is the process by which two or more protons and neutrons combine to form a single nucleus. It is a very important process in the universe, as it is the source of energy for stars and galaxies. Nuclear fusion is also a key process in the production of energy for nuclear power plants."
inputs = tokenizer(question, answer, return_tensors='pt')
inputs = inputs.to("cuda")
score = rank_model(**inputs).logits[0].detach()
print(score)

/home/charlie/Documents/untapped/ml-flow/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([2.2720], device='cuda:0')


In [2]:
import pandas as pd

df = pd.read_excel("data/loss_pairs_v3.ods", index_col=0).iloc[:4]

#for i in range(1,6):
#    df = pd.concat([df, pd.read_excel(f"data/loss_pairs_epoch_{i}.ods", index_col=0)])

scores = []
for prompt, completion in df[['prompt', 'completion']].values:
    inputs = tokenizer(prompt, completion, return_tensors='pt')
    inputs = inputs.to("cuda")
    score = rank_model(**inputs).logits[0].detach()
    scores.append(float(score))

print(scores)

[5.101278305053711, 2.1163647174835205, 2.9659976959228516, 6.296064853668213]


In [6]:
df['reward'] = scores
df.to_csv("output/test.csv")
df.head()

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
import pandas as pd

df = pd.read_excel("data/loss_pairs_v3.ods", index_col=0).iloc[:4]

batch = df[['prompt', 'completion']].values
batch = [(row[0], row[1]) for row in batch]
inputs = tokenizer.batch_encode_plus(batch, return_tensors='pt', truncation=True, padding="max_length", max_length=80).to('cuda')
scores = rank_model(**inputs)
print(scores)